This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [2]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
#from nltk.sem.drt import DrtParser

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Axioms

### Lattice axioms

In [3]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

### Bounded lattice axioms

In [4]:
bot = read_expr('Or(x, Bot) = x')
top = read_expr('And(x, Top) = x')

bounded_lattice = lattice + [bot, top]

### Distributive lattice axioms

In [5]:
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y), And(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

### Complete lattice axioms

In [6]:
uniq_sum = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)))')
uniq_product = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = z) & all y.((all x.(P(x) -> And(x, y) = y)) -> And(y,z) = y)))')

### Boolean algebra axioms

In [7]:
compl = read_expr('all x.(exists y. (And(x, y) = Bot & Or(x, y) = Top))')

#sectional complement
sec_compl = read_expr('all x. (all y. (exists z. (Or(x, z) = y & And(x, z) = Bot)))')

boolean_algebra = dist_bounded_lattice + [uniq_sum, uniq_product, compl]

### Exclusion axioms

In [8]:
excl_cumm = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(Or(x1, x2), Or(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [9]:
#define conflict:
confl_def = read_expr('confl(x, y) <-> exists z1.(And(z1, x) = z1 & exists z2.(And(z2, y) = z2 & Excl(z1, z2)))')

#define possibilities:
int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(And(y, z) = y & int_poss(z) & -(confl(z,x))))')

poss_def = read_expr('poss(x) <-> -confl(x, x)')

#define world:
#world_def = read_expr('world(x) <-> (int_poss(x) & all y.((And(x, y) = y) -> -int_poss(y)))')

world_def = read_expr('world(x) <-> (poss(x) & all y.((And(x, y) = x) -> -poss(y)))')

#actual world
#actual_world = read_expr('world(actual_world)')

#define actual:
#actual_def = read_expr('actual(x) <-> And(x, actual_world) = x')

#Harmony
#poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

poss_harm = read_expr('all x.(all y. ((world(x) & -confl(x, y)) -> poss(x)))')

#Nirvana
poss_nirv = read_expr('all x.(-confl(Bot, x))')

#Rashōmon
#poss_rshm = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(And(x,y)))')
poss_rshm = read_expr('all x y.((poss(x) & poss(y) & -confl(x, y)) -> poss(And(x,y)))')


#Cosmopolitanism
#poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & And(x, y) = y))')

poss_cosm = read_expr('all x.(poss(x) -> exists y.(world(y) & And(x, y) = x))')

poss_ax = [poss_harm, poss_nirv, poss_rshm, poss_cosm]

#Construct event frame
#choice point: definition of possibility
#choice point: formulation of harmony

event_space = boolean_algebra + [excl_cumm, excl_symm, confl_def, poss_def, world_def]

event_frame = event_space + poss_ax

## Theorems

### Plenitude

In [10]:
#check plenitude
#goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')

goal = read_expr('all x.(poss(x) <-> (exists y.(-confl(x, y) & world(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2717 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:19:45 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 43.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
10 (all x -confl(Bot,x)).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].


### Distributivity of actuality

In [11]:
#check distributivity of actuality
goal = read_expr('all x.((exists y. actual(y) & And(x, y) = x) -> actual(x))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2719 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:19:49 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 43.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
10 (all x -confl(Bot,x)).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].


In [12]:
#Find counterexample for distributivity of actuality - fail

goal = read_expr('all x.((exists y. actual(y) & And(x, y) = x) -> actual(x))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

False

### Cumulativity of actuality

In [13]:
#check cumulativity of actuality
goal = read_expr('all P.((all x.(P(x) -> actual(x))) -> (exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)) & actual(z)))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2730 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:20:17 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 45.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
10 (all x -confl(Bot,x)).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
15 -world(x) | poss(x).  [clausify(8)].
16 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].


In [14]:
#find counterexample for cumulativity of actuality - fail

goal = read_expr('all P.((all x.(P(x) -> actual(x))) -> (exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)) & actual(z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

False

### Manichaeism

In [15]:
#check Manichaeism
goal = read_expr('all x.((world(x) -> all y.(And(x, y) = y | confl(x, y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2738 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:20:35 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 13.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 35.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
13 (all x (world(x) -> (all y (And(x,y) = y | confl(x,y))))).  [goal].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
23 -poss(x) 

### Classicality of conjunction

In [16]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & actual(x))) & (exists y. (Q(y) & actual(y)))) <-> exists y exists z.(actual(y) & actual(z) & P(y) & Q(z) & actual(Or(y, z)))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2743 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:20:44 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 62.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
10 (all x -confl(Bot,x)).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
15 -world(x) | poss(x).  [clausify(8)].
16 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].


In [17]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & actual(x))) & (exists y. (Q(y) & actual(y)))) <-> exists y exists z.(actual(y) & actual(z) & P(y) & Q(z) & actual(Or(y, z)))')
mb = MaceCommand(goal, assumptions = event_frame)
mb.build_model()

False

### Classicality of disjunction

In [18]:
#check classicality of disjunction
goal = read_expr(r'(exists x. (actual(x) & (P(x) | Q(x)))) <-> ((exists x. (actual(x) & P(x))) & (exists y. (actual(y) & Q(y))))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2750 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:20:58 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 50.


7 poss(x) <-> -confl(x,x).  [assumption].
8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
10 (all x -confl(Bot,x)).  [assumption].
12 (all x (poss(x) -> (exists y (world(y) & And(x,y) = x)))).  [assumption].
15 -world(x) | poss(x).  [clausify(8)].
16 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].


### No Gaps

In [20]:
#define negating a set
def negate(x, P):
    return '(all x1. (P(x1) -> (exists x2. And(x,x2) = x2 & Excl(x1, x2))) & (all x3. ((And(x3, x) = x3 & (-exists x4. (And(x, x4) = x4 & And(x3, x4) = x4))) -> (exists x5. (P(x5) & Excl(x5, x3))))) & all x6 all x7. ((x6 != x7) -> (exists x8 exists x9.(And(x8, x) = x8 & And(x9, x) = x9 & x8!=x9 & Excl(x6, x8) & Excl(x7, x9)))))'

#check no gaps
goal = read_expr('(-exists x. (actual(x) & P(x))) <-> (exists y. (actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2406 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:04:57 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 59.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
16 -world(x) | poss(x).  [clausify(8)].
17 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
19 world(actual_world).  [assumption].
23 And(x,x) = x.  [assumption].
103 poss(actual_world).  [resolve(19,a,16,a)].
104 And(actual_world,x) != actual_world | -p

### No Gluts

In [21]:
#check no gluts
goal = read_expr('-((exists x. actual(x) & P(x)) & (exists y. actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_frame)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2408 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:05:00 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 45.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
25 -world(x) | poss(x).  [clausify(8)].
26 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
28 world(actual_world).  [assumption].
36 And(x,x) = x.  [assumption].
93 poss(actual_world).  [resolve(28,a,25,a)].
94 And(actual_world,x) != actual_world | -pos

## Independence of axioms

In [36]:
#Harmony is derivable from other possibility axioms

goal = poss_harm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_harm)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2574 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:13:24 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 2.000.
% Given clauses 0.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
13 (all x all y (world(x) & -confl(x,y) -> poss(x))).  [goal].
19 -world(x) | poss(x).  [clausify(8)].
24 world(c1).  [deny(13)].
61 -poss(c1).  [deny(13)].
72 poss(c1).  [resolve(24,a,19,a)].
73 $F.  [copy(72),unit_del(a,61)].

============================== 

In [44]:
#Nirvana is derivable from other possibility axioms (no counterexample found)
goal = poss_nirv
mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == poss_nirv)])
mb.build_model()

False

In [39]:
goal = poss_nirv
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_nirv)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2595 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:14:27 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 32.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
22 world(actual_world).  [assumption].
28 And(x,x) = x.  [assumption].
66 poss(actual_world).  [resolve(22,a,19,a)].
67 And(actual_world,x) != actual_world | -pos

In [40]:
#Rashōmon is derivable from other possibility axioms
goal = poss_rshm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_rshm)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2602 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:14:41 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 35.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
22 world(actual_world).  [assumption].
28 And(x,x) = x.  [assumption].
69 poss(actual_world).  [resolve(22,a,19,a)].
70 And(actual_world,x) != actual_world | -pos

In [41]:
#Cosmopolitan is not derivable from other possibility axioms (counterexample found)
goal = poss_cosm
mb = MaceCommand(goal, assumptions = [ax for ax in event_frame if not (ax == poss_cosm)])
mb.build_model()

False

In [42]:
#Cosmopolitan is derivable from other possibility axioms
goal = poss_cosm
prover = Prover9Command(goal, assumptions = [ax for ax in event_frame if not (ax == poss_cosm)], timeout = 1000)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 2608 was started by guzzler on 10-17-18-2.dynapool.wireless.nyu.edu,
Fri Jul  8 14:14:54 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 32.


8 world(x) <-> poss(x) & (all y (And(x,y) = x -> -poss(y))).  [assumption].
19 -world(x) | poss(x).  [clausify(8)].
20 -world(x) | And(x,y) != x | -poss(y).  [clausify(8)].
22 world(actual_world).  [assumption].
28 And(x,x) = x.  [assumption].
66 poss(actual_world).  [resolve(22,a,19,a)].
67 And(actual_world,x) != actual_world | -pos